In [8]:
import requests
import time
import pandas as pd

In [2]:
def poll_job(s, redash_url, job):
    # TODO: add timeout
    while job['status'] not in (3, 4):
        response = s.get('{}/api/jobs/{}'.format(redash_url, job['id']))
        job = response.json()['job']
        time.sleep(1)

    if job['status'] == 3:
        return job['query_result_id']

    return None

In [3]:
def get_fresh_query_result(redash_url, query_id, api_key, params):
    s = requests.Session()
    s.headers.update({'Authorization': 'Key {}'.format(api_key)})

    response = s.post('{}/api/queries/{}/refresh'.format(redash_url, query_id), params=params)
    if response.status_code != 200:
        raise Exception('Refresh failed.')

    result_id = poll_job(s, redash_url, response.json()['job'])

    if result_id:
        response = s.get('{}/api/queries/{}/results/{}.json'.format(redash_url, query_id, result_id))
        if response.status_code != 200:
            raise Exception('Failed getting results.')
    else:
        raise Exception('Query execution failed.')

    return pd.DataFrame(response.json()['query_result']['data']['rows'])

country_list = ["id","my","sg","ph","th"]

In [4]:
def month_performance():
    initial_params = {'p_country': 'tw'}
    final_result = get_fresh_query_result('http://10.12.5.53', 994, 'PrsLn6Mf09MuBxBTrAEeRdT3gyqKzbG20obScoEV', initial_params)
    for i in country_list:
        params = {'p_country': i}
        result = get_fresh_query_result('http://10.12.5.53', 994, 'PrsLn6Mf09MuBxBTrAEeRdT3gyqKzbG20obScoEV', params)
        final_result = final_result.append(result)
    final_result = final_result.rename(columns={"shopid":"Child ShopID"})
    final_result["Child ShopID"] = final_result["Child ShopID"].convert_objects(convert_numeric=True)
    return final_result

In [9]:
month_performance()

C:\Users\Enzo.kuang\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,m-1_gmv,m-1_gross_orders,m-1_live_skus,m-1_skus,mtd_gmv,mtd_gross_orders,mtd_live_skus,mtd_skus,Child ShopID
0,16127463.0,26554,759,1968,11934483.0,19018,905,1313,20162694
1,9962594.0,19372,1142,1853,6082893.0,11480,998,1082,9469128
2,6714455.0,15842,1338,1397,4468247.0,9994,462,485,9636377
3,6272114.0,10604,80,1027,5394869.0,9384,16,175,31871153
4,9706915.0,15204,399,1152,5608126.0,8814,732,989,23316439
5,6209446.0,10170,347,429,4570923.0,7349,328,341,26493322
6,5576664.0,11290,1759,1826,3646123.0,6962,787,810,9371163
7,7105751.0,12208,253,1214,3443424.0,6147,451,924,23318431
8,3446269.0,6092,428,434,2666341.0,4323,201,210,12029472
9,3854798.0,6143,482,517,2603089.0,4230,1050,1066,23487015
